In [44]:
#imports

import os
# This line changes tensorflow log level and suppresses warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.framework.ops import EagerTensor
from tensorflow.python.ops.resource_variable_ops import ResourceVariable
import time

In [45]:
# Data Import
train_dataset = h5py.File('datasets/train_signs.h5', "r")
test_dataset = h5py.File('datasets/test_signs.h5', "r")

In [46]:
# Data Import Check
train_dataset['train_set_x']

<HDF5 dataset "train_set_x": shape (1080, 64, 64, 3), type "|u1">

In [47]:
# Data Preparation
x_train = tf.data.Dataset.from_tensor_slices(train_dataset['train_set_x'])
y_train = tf.data.Dataset.from_tensor_slices(train_dataset['train_set_y'])

x_test = tf.data.Dataset.from_tensor_slices(test_dataset['test_set_x'])
y_test = tf.data.Dataset.from_tensor_slices(test_dataset['test_set_y'])

In [48]:
# Data Type Check
type(x_train)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [49]:
# Train and Test Data spec check
print('x_train spec: ', x_train.element_spec)
print('x_test spec: ', x_test.element_spec)
print('y_train spec: ', y_train.element_spec)
print('y_test spec: ', y_test.element_spec)

x_train spec:  TensorSpec(shape=(64, 64, 3), dtype=tf.uint8, name=None)
x_test spec:  TensorSpec(shape=(64, 64, 3), dtype=tf.uint8, name=None)
y_train spec:  TensorSpec(shape=(), dtype=tf.int64, name=None)
y_test spec:  TensorSpec(shape=(), dtype=tf.int64, name=None)


In [50]:
# Verifying x_train is iterable or not
print(next(iter(x_train)))

tf.Tensor(
[[[227 220 214]
  [227 221 215]
  [227 222 215]
  ...
  [232 230 224]
  [231 229 222]
  [230 229 221]]

 [[227 221 214]
  [227 221 215]
  [228 221 215]
  ...
  [232 230 224]
  [231 229 222]
  [231 229 221]]

 [[227 221 214]
  [227 221 214]
  [227 221 215]
  ...
  [232 230 224]
  [231 229 223]
  [230 229 221]]

 ...

 [[119  81  51]
  [124  85  55]
  [127  87  58]
  ...
  [210 211 211]
  [211 212 210]
  [210 211 210]]

 [[119  79  51]
  [124  84  55]
  [126  85  56]
  ...
  [210 211 210]
  [210 211 210]
  [209 210 209]]

 [[119  81  51]
  [123  83  55]
  [122  82  54]
  ...
  [209 210 210]
  [209 210 209]
  [208 209 209]]], shape=(64, 64, 3), dtype=uint8)


In [51]:
# Normalize Image
def normalize(image):
    image = tf.cast(image, tf.float32) / 256.0
    image = tf.reshape(image, [-1, 1])
    return image

In [52]:
new_train = x_train.map(normalize)
new_test = x_test.map(normalize)

In [53]:
# Train and Test Data spec check
print('new_train spec: ', new_train.element_spec)
print('new_test spec: ', new_test.element_spec)

new_train spec:  TensorSpec(shape=(12288, 1), dtype=tf.float32, name=None)
new_test spec:  TensorSpec(shape=(12288, 1), dtype=tf.float32, name=None)


In [54]:
print(next(iter(new_train)))

tf.Tensor(
[[0.88671875]
 [0.859375  ]
 [0.8359375 ]
 ...
 [0.8125    ]
 [0.81640625]
 [0.81640625]], shape=(12288, 1), dtype=float32)


In [55]:
# Linear Function using Tensorflow
def linear_function():
    # Y = WX + b

    np.random.seed(1)

    X = tf.constant(np.random.randn(3, 1))
    W = tf.constant(np.random.randn(4, 3))
    b = tf.constant(np.random.randn(4, 1))
    Y = tf.add(tf.matmul(W, X), b)

    return Y

In [56]:
result = linear_function()
print('basic linear function result: ', result)

basic linear function result:  tf.Tensor(
[[-2.15657382]
 [ 2.95891446]
 [-1.08926781]
 [-0.84538042]], shape=(4, 1), dtype=float64)


In [57]:
# Sigmoid using tensorflow
def sigmoid(z):
    z = tf.cast(z, tf.float32)
    a = tf.keras.activations.sigmoid(z)
    return a

In [58]:
result = sigmoid(-1)
print ("type: " + str(type(result)))
print ("dtype: " + str(result.dtype))
print ("sigmoid(-1) = " + str(result))
print ("sigmoid(0) = " + str(sigmoid(0.0)))
print ("sigmoid(12) = " + str(sigmoid(12)))

type: <class 'tensorflow.python.framework.ops.EagerTensor'>
dtype: <dtype: 'float32'>
sigmoid(-1) = tf.Tensor(0.26894143, shape=(), dtype=float32)
sigmoid(0) = tf.Tensor(0.5, shape=(), dtype=float32)
sigmoid(12) = tf.Tensor(0.9999938, shape=(), dtype=float32)


In [59]:
# One hot matrix in tensorflow
def one_hot_matrix(label, depth=8):

    one_hot = tf.one_hot(label, depth, axis=0)
    one_hot = tf.reshape(one_hot, [depth, 1])
    
    return one_hot

In [60]:
label = tf.constant(1)
depth = 10
result = one_hot_matrix(label, depth)
print(result)

tf.Tensor(
[[0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float32)


In [61]:
# Using one hot matrix
new_y_test = y_test.map(one_hot_matrix)
new_y_train = y_train.map(one_hot_matrix)

In [62]:
print('new_y_test: ', next(iter(new_y_test)))
print('new_y_train: ', next(iter(new_y_train)))

new_y_test:  tf.Tensor(
[[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(8, 1), dtype=float32)
new_y_train:  tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]], shape=(8, 1), dtype=float32)


In [63]:
# Parameters Initialization using tf
def initialize_parameters():
    initializer = tf.keras.initializers.GlorotNormal(seed=1)
    W1 = tf.Variable(initializer(shape=(25, 12288)))
    b1 = tf.Variable(initializer(shape=(25, 1)))
    W2 = tf.Variable(initializer(shape=(12, 25)))
    b2 = tf.Variable(initializer(shape=(12, 1)))
    W3 = tf.Variable(initializer(shape=(6, 12)))
    b3 = tf.Variable(initializer(shape=(6, 1)))

    params = {
        'W1': W1,
        'b1': b1,
        'W2': W2,
        'b2': b2,
        'W3': W3,
        'b3': b3
    }
    return params

In [64]:
params = initialize_parameters()

## Neural Network using Tensorflow

In [65]:
# forward_propogation
@tf.function
def forward_propogation(X, params):

    # model: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR

    W1 = params['W1']
    b1 = params['b1']
    W2 = params['W2']
    b2 = params['b2']
    W3 = params['W3']
    b3 = params['b3']

    Z1 = tf.add(tf.matmul(W1, X), b1)
    A1 = tf.keras.activations.relu(Z1)
    Z2 = tf.add(tf.matmul(W2, A1), b2)
    A2 = tf.keras.activations.relu(Z2)
    Z3 = tf.add(tf.matmul(W3, A2), b3)

    return Z3

In [66]:
# Forward Propagation test
for batch in new_train:
    forward_pass = forward_propogation(batch, params)
    print(forward_pass)

tf.Tensor(
[[-0.13082135]
 [ 0.21228778]
 [ 0.7050023 ]
 [-1.1224036 ]
 [-0.20386732]
 [ 0.9526217 ]], shape=(6, 1), dtype=float32)
tf.Tensor(
[[ 0.1430974 ]
 [-0.02823615]
 [ 0.64807725]
 [-0.9303614 ]
 [-0.33988   ]
 [ 0.93866   ]], shape=(6, 1), dtype=float32)
tf.Tensor(
[[-0.04745959]
 [ 0.10759842]
 [ 0.6819784 ]
 [-0.9367415 ]
 [-0.261388  ]
 [ 0.89727867]], shape=(6, 1), dtype=float32)
tf.Tensor(
[[ 0.06029473]
 [ 0.05436924]
 [ 0.6011406 ]
 [-0.9858722 ]
 [-0.31344664]
 [ 0.9050338 ]], shape=(6, 1), dtype=float32)
tf.Tensor(
[[ 0.00690911]
 [ 0.1209895 ]
 [ 0.6118636 ]
 [-0.96064615]
 [-0.23491931]
 [ 0.8752539 ]], shape=(6, 1), dtype=float32)
tf.Tensor(
[[ 0.02325378]
 [ 0.05176771]
 [ 0.6729045 ]
 [-0.9559515 ]
 [-0.2818824 ]
 [ 0.9047475 ]], shape=(6, 1), dtype=float32)
tf.Tensor(
[[ 0.08214916]
 [ 0.03821006]
 [ 0.60222423]
 [-0.9108583 ]
 [-0.30919802]
 [ 0.81098473]], shape=(6, 1), dtype=float32)
tf.Tensor(
[[-0.01605102]
 [ 0.08654475]
 [ 0.61092365]
 [-1.055867  ]
 [-0.

tf.Tensor(
[[-0.04514437]
 [ 0.07678327]
 [ 0.6113426 ]
 [-1.0975732 ]
 [-0.27322516]
 [ 0.8625325 ]], shape=(6, 1), dtype=float32)
tf.Tensor(
[[-0.19160861]
 [ 0.13880551]
 [ 0.6986897 ]
 [-1.0105692 ]
 [-0.44207007]
 [ 0.76820636]], shape=(6, 1), dtype=float32)
tf.Tensor(
[[-0.17576012]
 [-0.09609649]
 [ 0.6909901 ]
 [-0.9384085 ]
 [-0.51323974]
 [ 0.7635832 ]], shape=(6, 1), dtype=float32)
tf.Tensor(
[[-0.02548277]
 [ 0.01579639]
 [ 0.6846498 ]
 [-1.0315487 ]
 [-0.32406193]
 [ 0.9154924 ]], shape=(6, 1), dtype=float32)
tf.Tensor(
[[-0.0445325 ]
 [ 0.04053843]
 [ 0.6773903 ]
 [-1.035063  ]
 [-0.33042747]
 [ 0.8446814 ]], shape=(6, 1), dtype=float32)
tf.Tensor(
[[ 0.00445551]
 [ 0.11237928]
 [ 0.6489879 ]
 [-0.7697273 ]
 [-0.31262323]
 [ 0.7374654 ]], shape=(6, 1), dtype=float32)
tf.Tensor(
[[ 0.01684041]
 [ 0.22638184]
 [ 0.78933924]
 [-0.7750089 ]
 [-0.16080731]
 [ 1.0989475 ]], shape=(6, 1), dtype=float32)
tf.Tensor(
[[ 0.12524474]
 [ 0.02537176]
 [ 0.6549487 ]
 [-1.0102351 ]
 [-0.